In [5]:
!pip install flask torch pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [6]:
#Create the ngrok account
!ngrok authtoken 2vXmUOUCmNtiJk9dTT4vLL3VdHg_tnWLNfdx9P85sbZCKuww

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
from flask import Flask, request, jsonify
import torch
from LSTM import LSTM_classifier
from preprocessing import accepted_chars
import pickle
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

app = Flask(__name__)

# Load LSTM model
try:
    lstm_model = LSTM_classifier(d_model=8, hidd_size=32, lstm_depth=2, num_answers=2)
    lstm_model.load_state_dict(torch.load('lstm_url_classifier.pth'))
    lstm_model.eval()
    print("LSTM model loaded successfully")
except Exception as e:
    print(f"Error loading LSTM model: {e}")

# Load Random Forest model
try:
    with open('final_rf_model.pkl', 'rb') as f:
        rf_model = pickle.load(f)
    print("Random Forest model loaded successfully")
except Exception as e:
    print(f"Error loading RF model: {e}")

# Placeholder vectorizer (adjust based on your RF model)
vectorizer = TfidfVectorizer(max_features=1000)
# Fit with dummy data (replace with actual training senders or load saved vectorizer)
dummy_senders = ['user@example.com', 'spam@bad.com']
vectorizer.fit(dummy_senders)
# If you have a saved vectorizer:
# try:
#     with open('tfidf_vectorizer.pkl', 'rb') as f:
#         vectorizer = pickle.load(f)
#     print("Vectorizer loaded successfully")
# except Exception as e:
#     print(f"Error loading vectorizer: {e}")

def preprocess_sender(sender):
    try:
        # Extract email address
        email_match = re.search(r'<([^>]+)>', sender) or re.search(r'[\w\.-]+@[\w\.-]+', sender)
        email = email_match.group(1) if email_match else sender.lower()
        features = vectorizer.transform([email])
        print(f"Preprocessed sender: {email}")
        return features
    except Exception as e:
        print(f"Error preprocessing sender {sender}: {e}")
        return None

def predict_sender(sender):
    try:
        features = preprocess_sender(sender)
        if features is None:
            return "Non-Spam", 0.0
        prediction = rf_model.predict(features)[0]
        probability = rf_model.predict_proba(features)[0][prediction]  # Get prob for predicted class
        label = "Spam" if prediction == 1 else "Non-Spam"
        print(f"Sender: {sender}, Label: {label}, Prob: {probability}")
        return label, probability
    except Exception as e:
        print(f"Sender prediction error for {sender}: {e}")
        return "Non-Spam", 0.0

def predict_url(url, model):
    try:
        url = url.lower()
        if any(char not in accepted_chars for char in url):
            print(f"Invalid characters in URL: {url}")
            return "Benign", 0.0
        url_batch = [url]
        with torch.no_grad():
            output = model(url_batch)
            prediction = output.argmax(1).item()
            probability = output[0].max().item()
            label = "Benign" if prediction == 1 else "Malicious"
            print(f"URL: {url}, Raw output: {output}, Label: {label}, Prob: {probability}")
            return label, probability
    except Exception as e:
        print(f"URL prediction error for {url}: {e}")
        return "Benign", 0.0

@app.route('/predict_sender', methods=['POST'])
def predict_sender_endpoint():
    data = request.json
    sender = data.get('sender', '')
    print(f"Received /predict_sender request for: {sender}")
    label, prob = predict_sender(sender)
    return jsonify({'label': label, 'probability': prob})

@app.route('/predict_url', methods=['POST'])
def predict_url_endpoint():
    data = request.json
    url = data.get('url', '')
    print(f"Received /predict_url request for: {url}")
    label, prob = predict_url(url, lstm_model)
    return jsonify({'label': label, 'probability': prob})


@app.route('/predict_email', methods=['POST'])
def predict_email():
    data = request.json
    sender = data.get('sender', '')
    urls = data.get('urls', [])
    print(f"Received /predict_email request for sender: {sender}, urls: {urls}")

    # Sender prediction
    sender_label, sender_prob = predict_sender(sender)

    # URL predictions
    url_results = [predict_url(url, lstm_model) for url in urls]
    # Filter URLs with "Malicious" label and probability > 0.9
    malicious_urls = [
        (url, prob)
        for url, (label, prob) in zip(urls, url_results)
        if label == "Malicious" and prob > 0.9
    ]

    # New condition: Email is spam if sender is "Spam" or there are at least 3 high-confidence malicious URLs
    is_spam = (sender_label == "Spam") or (len(malicious_urls) >= 3)

    print(f"Sender Label: {sender_label}, Malicious URLs: {len(malicious_urls)}, Is Spam: {is_spam}")
    return jsonify({
        'is_spam': is_spam,
        'sender_label': sender_label,
        'sender_probability': sender_prob,
        'malicious_urls': [{'url': url, 'probability': prob} for url, prob in malicious_urls]
    })


@app.route('/test', methods=['GET'])
def test():
    return jsonify({'status': 'Server is running'})

if __name__ == '__main__':
    from pyngrok import ngrok
    public_url = ngrok.connect(5000).public_url
    print(f"Server running at: {public_url}")
    app.run(port=5000)

LSTM model loaded successfully
Random Forest model loaded successfully
Server running at: https://8e4a-34-56-46-4.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Received /predict_email request for sender: AccioJob via Naukri <info@naukri.com>, urls: ['https://cdn.ampproject.org/v0.js', 'https://cdn.ampproject.org/v0/amp-form-0.1.js', 'https://cdn.ampproject.org/v0/amp-bind-0.1.js', 'https://logs.naukri.com/collectorapi/v1/uba?data=%7B%22deviceType%22%3A%22WEB%22%2C%22additionalParams%22%3A%7B%7D%2C%22campaignId%22%3Anull%2C%22isAmp%22%3A%221%22%2C%22communicationId%22%3A%2267e4f1fa96c6f8c76c8aa74c%22%2C%22communicationType%22%3A%22mail%22%2C%22journeyId%22%3Anull%2C%22userId%22%3A%22231834814%22%2C%22eventCode%22%3A%22ccs_top_applicant_revival_mail%22%2C%22encEmail%22%3A%22a23fb0d4f22151067d67eb4c968cdcae6a7d55580dd6f6d8d73cd87d3b1cd9291edd7a843282321a%22%2C%22mailType%22%3A%22ccs_top_applicant_revival_mail%22%2C%22mailerId%22%3A%2220250327492%22%2C%22requestId%22%3A%2267e4edf26e293d6d72498503%22%2C%22appId%22%3A%22120%22%2C%22segmentId%22%3Anull%2C%22tenantId%22%3A%221%22%2C%22eventName%22%3A%22communicationOpen%22%2C%22parentActivityId%22%3A

INFO:werkzeug:127.0.0.1 - - [16/Apr/2025 15:47:51] "POST /predict_email HTTP/1.1" 200 -


URL: https://cm.naukri.com?data=%7b%22devicetype%22%3a%22web%22%2c%22additionalparams%22%3a%7b%7d%2c%22communicationid%22%3a%2267e4f1fa96c6f8c76c8aa74c%22%2c%22clickposition%22%3a12%2c%22communicationtype%22%3a%22mail%22%2c%22userid%22%3a%22f3b090e4ccb7d81e9a4ff92e567bd2f5adf4ac47e4485ce35c4f31944917d4da%22%2c%22eventcode%22%3a%22ccs_top_applicant_revival_mail%22%2c%22encemail%22%3a%22a23fb0d4f22151067d67eb4c968cdcae6a7d55580dd6f6d8d73cd87d3b1cd9291edd7a843282321a%22%2c%22mailtype%22%3a%22ccs_top_applicant_revival_mail%22%2c%22mailerid%22%3a%2220250327492%22%2c%22appid%22%3a120%2c%22tenantid%22%3a%221%22%2c%22eventname%22%3a%22communicationclick%22%7d&amp;redirect=https%3a%2f%2fcompany.naukri.com%2flanding-page%2ffakejobtrend%2fnew%2findex.html%3futm_campaign%3dja_stress_top_applicant%26utm_medium%3demail%26utm_source%3dsecurity%26utm_content%3dccs_top_applicant_revival_mail%26utm_term%3d20250327492, Raw output: tensor([[0.8088, 0.1912]]), Label: Malicious, Prob: 0.8088297843933105
URL

...

Received /predict_email request for sender: Myntra <updates@myntra.com>, urls: ['https://fonts.googleapis.com/css?family=Gothic+A1:400,700', 'https://myntrabuyers.myntramailers.com/vtrack?clientid=96861&ul=', 'https://assets.myntassets.com/assets/images/2025/APRIL/16/2BlKfY94_59f44b24a61d4b5c84224a7a2aeaae70.jpg', 'https://myntrabuyers.myntramailers.com/vtrack?clientid=96861&ul=', 'https://assets.myntassets.com/assets/images/2025/APRIL/16/YgNxMqEp_49840c2ffda74dbdb635f9f4c87f8e06.jpg', 'https://myntrabuyers.myntramailers.com/vtrack?clientid=96861&ul=', 'https://assets.myntassets.com/assets/images/2025/APRIL/16/XVf30TmM_51d196a3dafc43bb8e1e2fc4129feed2.gif', 'https://myntrabuyers.myntramailers.com/vtrack?clientid=96861&ul=', 'https://assets.myntassets.com/assets/images/2025/APRIL/16/8R2ADHzL_79ffe9428b1d4014baeb59086f938059.jpg', 'https://myntrabuyers.myntramailers.com/vtrack?clientid=96861&ul=', 'https://assets.myntassets.com/assets/images/2025/APRIL/12/xKVrxwTq_952537024eee448e9caef37

INFO:werkzeug:127.0.0.1 - - [16/Apr/2025 16:19:54] "POST /predict_email HTTP/1.1" 200 -


Received /predict_email request for sender: Grafana Labs Team <update@grafana.com>, urls: ['https://go.grafana.com/MzU2LVlGRy0zODkAAAGZ3lOVsNlZJgoA5HjyUm6JuNlwz0jTZ25x73eCebu7NvOgdjsTk3DdToIl3lLT3UjR_Gcv2bo=', 'https://go.grafana.com/MzU2LVlGRy0zODkAAAGZ3lOVsMiFCGT3eiNMj2NeYZdDTpDCEDLf1w6gJ1ycuZhsp0iflexid_qT_8Iqjes2x8SKiVw=', 'https://go.grafana.com/MzU2LVlGRy0zODkAAAGZ3lOVsIQuW16BKn9uFdsPaDxILift1JHdf_WXBBovfqy_X_ixRm_EiZp4BVzT0qxFAAjRkwQ=', 'https://go.grafana.com/MzU2LVlGRy0zODkAAAGZ3lOVsBbPqwTm3akO43K4bAi4tl5NJFi5b-vhJPgvcMUMjboIQB_DiLJJot6Y9m6YZkFcYWs=', 'https://go2.grafana.com/user-preferences.html?mkt_tok=MzU2LVlGRy0zODkAAAGZ3lOVsCEDKXB6kivOpuljHHIyc7ozd4ukP7jfxmN3MxScfLdBaTxQma3mdDUU3pSgrOZLCVRGXP1c1hfFuWg8fRdei_KX9Eg5v9MN_54x_aJKGhk', 'https://go2.grafana.com/UnsubscribePage.html?mkt_unsubscribe=1&mkt_tok=MzU2LVlGRy0zODkAAAGZ3lOVsCEDKXB6kivOpuljHHIyc7ozd4ukP7jfxmN3MxScfLdBaTxQma3mdDUU3pSgrOZLCVRGXP1c1hfFuWg8fRdei_KX9Eg5v9MN_54x_aJKGhk.', 'http://www.w3.org/TR/xhtml1/DTD/xhtm

INFO:werkzeug:127.0.0.1 - - [16/Apr/2025 16:19:56] "POST /predict_email HTTP/1.1" 200 -


URL: https://www.linkedin.com/comm/games/tango/?lipi=urn%3ali%3apage%3aemail_email_network_conversations_01%3bvtv29srvrzasrg4lm5yl1w%3d%3d&amp;midtoken=aqhdtqiixeyb4w&amp;midsig=2vf_vwxe2mkhi1&amp;trk=eml-email_network_conversations_01-email~footer~play-0-tango&amp;trkemail=eml-email_network_conversations_01-email~footer~play-0-tango-null-l34ue9~m9k0u5cj~n-null-null&amp;eid=l34ue9-m9k0u5cj-n&amp;otptoken=mtmwnze5ztuxmjjiy2rjmwiymmywzmvindexywuzymm4zwnhzdq0ndkwywq4njyzn2jjzja5nmq0otvknwzmn2yzzdjkmwu5nwjmowi5zmi0mge1ytjhnthhmje2mjviytjlmjk0otnknzbmntvkodg5nzy2oweyldesmq%3d%3d, Raw output: tensor([[0.6387, 0.3613]]), Label: Malicious, Prob: 0.6386795043945312
URL: https://www.linkedin.com/comm/games/queens/?lipi=urn%3ali%3apage%3aemail_email_network_conversations_01%3bvtv29srvrzasrg4lm5yl1w%3d%3d&amp;midtoken=aqhdtqiixeyb4w&amp;midsig=2vf_vwxe2mkhi1&amp;trk=eml-email_network_conversations_01-queens~games~hub~tap~target-0-entity~image&amp;trkemail=eml-email_network_conversations_01-queens~